In [1]:
%%capture
!pip install modisco-lite 


In [3]:
import h5py
import torch
import pandas as pd
import numpy as np
from tangermeme.utils import characters
from tangermeme.seqlet import tfmodisco_seqlets
from tangermeme.plot import plot_logo
from tangermeme.utils import one_hot_encode
from tangermeme.deep_lift_shap import deep_lift_shap, _nonlinear
import torch, torch.nn as nn, torch.fx as fx, copy
import copy
import matplotlib.pyplot as plt

In [4]:
#get one hots

def load_attributions_with_test_index(h5_file_path):
    """
    Load scaled foreground attributions while preserving original sequence indices
    """
    with h5py.File(h5_file_path, 'r') as f:
        orig_df = pd.DataFrame(pd.read_csv('/grid/wsbs/home_norepl/pmantill/Motif_Swap_Experiments/Replicating_Motif_Swap_Manuscript/experimental_library_generation/Binned_libraries/Hk_final_df.csv'))

        # Load the attribution data (shape: 8999, 3, 249, 4)
        all_attributions = f['attributions'][:]
        print(all_attributions.shape)
        
        # Load the original indices mapping (shape: 8999,)
        #original_indices = f['original_indices'][:]
        
        # Extract scaled foreground (index 1)
        scaled_foreground_attr = torch.tensor(all_attributions[:, 1, :, :], dtype=torch.float32)
        cluster_foreground_attr = torch.tensor(all_attributions[:, 0, :, :], dtype=torch.float32)
        average_background_attr = torch.tensor(all_attributions[:, 2, :, :], dtype=torch.float32)
        test_idx = orig_df["Unnamed: 0"]
        bin = orig_df["EvoAug_Score_Hk_bin"]
        real_score = orig_df["Real_Score_Hk"]
        pred = orig_df["EvoAug_Score_Hk"]
        seq = orig_df["Sequence"]

        df = pd.DataFrame(list(zip(scaled_foreground_attr, cluster_foreground_attr, average_background_attr, test_idx, bin, real_score, pred, seq)), columns=['Scaled_foreground', 'Cluster_foreground', 'Avg_background', 'test_idx', 'bin', 'real_score', 'evoaug_pred', "str_seq"])
        
        print(df.shape)
        
        return df

h5_file_path = '/grid/wsbs/home_norepl/pmantill/Motif_Swap_Experiments/Replicating_Motif_Swap_Manuscript/SEAM_analysis/Hk_SEAM_results/Hk_task1_attributions.h5'

df = load_attributions_with_test_index(h5_file_path)



(9000, 3, 249, 4)
(9000, 8)


In [5]:
df["ohe_seq"] = None

df

,Scaled_foreground,Cluster_foreground,Avg_background,test_idx,bin,real_score,evoaug_pred,str_seq,ohe_seq
0,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",287455,Low,-0.714099,-1.120561,CACATCCACCAGCTTACAAAAGTATGCGCCGACATATATATATAAA...,None
1,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",41729,Low,-0.242478,-1.112688,AGCAGGCATGTGGCAAGTAGCAAAACTAGCAGCACATCCATTCCAA...,None
2,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",348462,Low,-0.813635,-1.040152,TAGTTTAAGCCCTATTTATGTACGTCCGATGCGGTTGAAAATAAAA...,None
3,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",308155,Low,-0.550601,-1.076290,GCCGGAGACCGATGTGTGGACAAGGGCGGGTGCGGCGACGGCGGCC...,None
4,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",225128,Low,-1.405261,-1.026221,CAAGGGATATTTATTGGAACTTCTTATAGCGGGTTTTATGCTTTTA...,None
...,...,...,...,...,...,...,...,...,...
8995,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",1769,High,2.282502,3.611905,GGTGCACGGATAATGATGATGACTCAATTGACGTACAATGGCCAAC...,None
8996,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",25441,High,5.305958,2.338521,TGGTGTTAGAGGTGGAATTCATACGATTGGCTGATCTTGATGATGT...,None
8997,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",21153,High,3.870863,4.662454,AGTCATTTGTGGGTGCAGTTTCTGCGATTTTTCCGCCGTTACCTTC...,None
8998,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",31354,High,3.156617,2.349327,GCTGGCCCAGCACTGGCCTGGAATGACTGGCAGGCTCTCCATCCCT...,None


In [6]:
ohe_seqs = []
for i in range(len(df)):
    ohe_seq = one_hot_encode(df.iloc[i]["str_seq"])
    ohe_seqs.append(ohe_seq)

df["ohe_seq"] = ohe_seqs
    
df

,Scaled_foreground,Cluster_foreground,Avg_background,test_idx,bin,real_score,evoaug_pred,str_seq,ohe_seq
0,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",287455,Low,-0.714099,-1.120561,CACATCCACCAGCTTACAAAAGTATGCGCCGACATATATATATAAA...,"[[tensor(0, dtype=torch.int8), tensor(1, dtype..."
1,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",41729,Low,-0.242478,-1.112688,AGCAGGCATGTGGCAAGTAGCAAAACTAGCAGCACATCCATTCCAA...,"[[tensor(1, dtype=torch.int8), tensor(0, dtype..."
2,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",348462,Low,-0.813635,-1.040152,TAGTTTAAGCCCTATTTATGTACGTCCGATGCGGTTGAAAATAAAA...,"[[tensor(0, dtype=torch.int8), tensor(1, dtype..."
3,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",308155,Low,-0.550601,-1.076290,GCCGGAGACCGATGTGTGGACAAGGGCGGGTGCGGCGACGGCGGCC...,"[[tensor(0, dtype=torch.int8), tensor(0, dtype..."
4,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",225128,Low,-1.405261,-1.026221,CAAGGGATATTTATTGGAACTTCTTATAGCGGGTTTTATGCTTTTA...,"[[tensor(0, dtype=torch.int8), tensor(1, dtype..."
...,...,...,...,...,...,...,...,...,...
8995,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",1769,High,2.282502,3.611905,GGTGCACGGATAATGATGATGACTCAATTGACGTACAATGGCCAAC...,"[[tensor(0, dtype=torch.int8), tensor(0, dtype..."
8996,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",25441,High,5.305958,2.338521,TGGTGTTAGAGGTGGAATTCATACGATTGGCTGATCTTGATGATGT...,"[[tensor(0, dtype=torch.int8), tensor(0, dtype..."
8997,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",21153,High,3.870863,4.662454,AGTCATTTGTGGGTGCAGTTTCTGCGATTTTTCCGCCGTTACCTTC...,"[[tensor(1, dtype=torch.int8), tensor(0, dtype..."
8998,"[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...","[[tensor(0.), tensor(0.), tensor(0.), tensor(0...",31354,High,3.156617,2.349327,GCTGGCCCAGCACTGGCCTGGAATGACTGGCAGGCTCTCCATCCCT...,"[[tensor(0, dtype=torch.int8), tensor(0, dtype..."


In [7]:
# split by bin 

low_df = df[df["bin"] == 'Low']
mid_df = df[df["bin"] == 'Mid']
high_df = df[df["bin"] == 'High']

print(len(low_df), len(mid_df), len(high_df))

3000 3000 3000


In [8]:
# Fix typos and ensure consistent naming
low_ohe_list = low_df["ohe_seq"].values          # array of length N, each entry shape (L, 4)
low_att_list = low_df["Scaled_foreground"].values

mid_ohe_list = mid_df["ohe_seq"].values
mid_att_list = mid_df["Scaled_foreground"].values

high_ohe_list = high_df["ohe_seq"].values
high_att_list = high_df["Scaled_foreground"].values

# If entries are torch tensors, convert each to numpy float32
def to_numpy32(x):
    if isinstance(x, torch.Tensor):
        return x.detach().cpu().numpy().astype('float32')
    x = np.asarray(x)
    return x.astype('float32')

low_ohe = np.stack([to_numpy32(x) for x in low_ohe_list], axis=0).transpose(0, 2, 1)   # shape (N, L, 4)
low_att = np.stack([to_numpy32(x) for x in low_att_list], axis=0)   # shape (N, L, 4)

mid_ohe = np.stack([to_numpy32(x) for x in mid_ohe_list], axis=0).transpose(0, 2, 1) 
mid_att = np.stack([to_numpy32(x) for x in mid_att_list], axis=0)

high_ohe = np.stack([to_numpy32(x) for x in high_ohe_list], axis=0).transpose(0, 2, 1) 
high_att = np.stack([to_numpy32(x) for x in high_att_list], axis=0)

print(low_ohe.shape, high_att.shape)  # Expect (N_low, L, 4) and (N_high, L, 4)


(3000, 249, 4) (3000, 249, 4)


In [9]:
assert low_ohe.shape[-1] == 4 and low_att.shape[-1] == 4
assert low_ohe.shape == low_att.shape
assert mid_ohe.shape == mid_att.shape
assert high_ohe.shape == high_att.shape


In [10]:
import modiscolite
import numpy as np

def run_bin(ohe, att, window_size=20, flank=5):
    assert ohe.shape == att.shape and ohe.shape[-1] == 4
    ohe = ohe.astype('float32')
    att = att.astype('float32')
    pos_patterns, neg_patterns = modiscolite.tfmodisco.TFMoDISco(
        one_hot=ohe,
        hypothetical_contribs=att,
        sliding_window_size=window_size,
        flank_size=flank,
        verbose=True
    )
    return pos_patterns, neg_patterns

pos_low, neg_low = run_bin(low_ohe, low_att)   # (N_low, L, 4)
pos_mid, neg_mid = run_bin(mid_ohe, mid_att)   # (N_mid, L, 4)
pos_high, neg_high = run_bin(high_ohe, high_att)  # (N_high, L, 4)

Using 808 positive seqlets
Extracted 1589 negative seqlets
Using 4595 positive seqlets
Extracted 448 negative seqlets
Using 8380 positive seqlets


In [13]:
# Save each bin separately
modiscolite.io.save_hdf5('TF-Modisco-lite_results/Hk_low_bin_motifs.h5', pos_low, neg_low, 20)
modiscolite.io.save_hdf5('TF-Modisco-lite_results/Hk_mid_bin_motifs.h5', pos_mid, neg_mid, 20)  
modiscolite.io.save_hdf5('TF-Modisco-lite_results/Hk_high_bin_motifs.h5', pos_high, neg_high, 20)


In [ ]:
import os
import modiscolite.io
from modiscolite.util import MemeDataType

# Define bins and corresponding .h5 inputs
bins = {
    'low':  'TF-Modisco-lite_results/h5py_files/Hk_low_bin_motifs.h5',
    'mid':  'TF-Modisco-lite_results/h5py_files/Hk_mid_bin_motifs.h5',
    'high': 'TF-Modisco-lite_results/h5py_files/Hk_high_bin_motifs.h5'
}

# Base directories
report_dir    = 'TF-Modisco-lite_results'
meme_dir = os.path.join(report_dir, 'meme_files')
os.makedirs(meme_dir, exist_ok=True)

# Loop over bins and datatypes
for bin_name, h5_file in bins.items():
    for dtype in [MemeDataType.PFM, MemeDataType.CWM]:
        suffix = 'pwms' if dtype == MemeDataType.PFM else 'cwms'
        meme_file = os.path.join(
            meme_dir, f'Hk_{bin_name}_{suffix}.meme'
        )
        print(f'Writing {suffix.upper()} for {bin_name}-bin → {meme_file}')
        modiscolite.io.write_meme_from_h5(
            filename=h5_file,
            datatype=dtype,
            output_filename=meme_file,
            is_quiet = True
        )


Writing PWMS for low-bin → TF-Modisco-lite_results/meme_files/Hk_low_pwms.meme
Writing CWMS for low-bin → TF-Modisco-lite_results/meme_files/Hk_low_cwms.meme
Writing PWMS for mid-bin → TF-Modisco-lite_results/meme_files/Hk_mid_pwms.meme
Writing CWMS for mid-bin → TF-Modisco-lite_results/meme_files/Hk_mid_cwms.meme
Writing PWMS for high-bin → TF-Modisco-lite_results/meme_files/Hk_high_pwms.meme
Writing CWMS for high-bin → TF-Modisco-lite_results/meme_files/Hk_high_cwms.meme


In [ ]:
##FIMO-lite

In [ ]:
##TOMTOM-lite